## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
#Import Sys to grab config
import sys
# Import API key
sys.path.insert(1, '..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsh Harbour,BS,26.5412,-77.0636,79.83,71,0,12.37,clear sky
1,1,Yellowknife,CA,62.4560,-114.3525,30.72,80,100,27.63,overcast clouds
2,2,Namatanai,PG,-3.6667,152.4333,80.28,83,99,10.18,overcast clouds
3,3,Albany,US,42.6001,-73.9662,62.46,32,65,4.00,broken clouds
4,4,Thompson,CA,55.7435,-97.8558,46.56,87,100,13.80,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsh Harbour,BS,26.5412,-77.0636,79.83,71,0,12.37,clear sky
2,2,Namatanai,PG,-3.6667,152.4333,80.28,83,99,10.18,overcast clouds
5,5,Kavieng,PG,-2.5744,150.7967,83.25,73,97,18.05,overcast clouds
9,9,Mahebourg,MU,-20.4081,57.7000,75.49,83,40,5.75,scattered clouds
11,11,Carnot,CF,4.9409,15.8665,81.88,69,39,2.59,scattered clouds
13,13,Rikitea,PF,-23.1203,-134.9692,76.48,79,44,22.68,light rain
14,14,Bambous Virieux,MU,-20.3428,57.7575,75.45,83,40,5.75,scattered clouds
18,18,Puerto Ayora,EC,-0.7393,-90.3518,78.76,79,9,12.37,clear sky
20,20,Merauke,ID,-8.4667,140.3333,79.75,92,96,10.71,overcast clouds
21,21,Sakakah,SA,29.9697,40.2064,78.37,12,23,8.86,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
count_nan = len(preferred_cities_df) - preferred_cities_df.count()
count_nan

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#count_nan = 0, nothing to drop
count_nan

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsh Harbour,BS,79.83,clear sky,26.5412,-77.0636,
2,Namatanai,PG,80.28,overcast clouds,-3.6667,152.4333,
5,Kavieng,PG,83.25,overcast clouds,-2.5744,150.7967,
9,Mahebourg,MU,75.49,scattered clouds,-20.4081,57.7000,
11,Carnot,CF,81.88,scattered clouds,4.9409,15.8665,
13,Rikitea,PF,76.48,light rain,-23.1203,-134.9692,
14,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
18,Puerto Ayora,EC,78.76,clear sky,-0.7393,-90.3518,
20,Merauke,ID,79.75,overcast clouds,-8.4667,140.3333,
21,Sakakah,SA,78.37,few clouds,29.9697,40.2064,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame, add location parameter
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name. - Converting blank name to NaN and then dropping NaN
hotel_df["Hotel Name"].replace("",np.nan, inplace = True)
hotel_df.dropna(subset=["Hotel Name"], inplace = True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsh Harbour,BS,79.83,clear sky,26.5412,-77.0636,Abaco Beach Resort
2,Namatanai,PG,80.28,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
5,Kavieng,PG,83.25,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
9,Mahebourg,MU,75.49,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
13,Rikitea,PF,76.48,light rain,-23.1203,-134.9692,People ThankYou


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))